# AI Wine Sommelier RAG

## Wine Review Indexing

https://www.kaggle.com/datasets/christopheiv/winemagdata130k

In [4]:
%pip install -Uq langchain langchain-openai langchain-pinecone langchain-community

Note: you may need to restart the kernel to use updated packages.


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()  # 현재 경로의 .env를 읽어 환경변수로 등록
os.environ['LANGSMITH_TRACING'] = 'true'  # LangSmith 트레이싱(추적) 기능 활성화
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')          # .env의 langsmith_key 값을 LANGSMITH_API_KEY 키로 설정
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")                # .env의 openai_key 값을 OPENAI_API_KEY 키로 설정
os.environ['PINECONE_API_KEY'] = os.getenv("pinecone_key")            # .env의 pinecone_key 값을 PINECONE_API_KEY 키로 설정

## Pinecone 테스트

In [6]:
# 데이터로드
from langchain_core.documents import Document

documents = [
    Document(page_content="LangChain은 LLM 기반 애플리케이션을 쉽게 만들 수 있는 프레임워크입니다.", metadata={"source": "https://langchain.com/docs", "author": "alice", "page": 1}),
    Document(page_content="ChromaDB는 오픈소스 벡터 데이터베이스입니다.", metadata={"source": "https://chromadb.org/intro", "license": "MIT", "date": "2024-07-01"}),
    Document(page_content="파이썬으로 AI 서비스를 개발할 수 있습니다.", metadata={"source": "https://pythonai.co.kr", "editor": "kim", "page": 7}),
    Document(page_content="LLM은 자연어 처리를 위한 대형 언어 모델을 의미합니다.", metadata={"source": "https://llmwiki.com/info", "author": "bob", "version": "v1.1"}),
    Document(page_content="RAG는 검색과 생성의 결합 방식을 제공합니다.", metadata={"source": "https://rag-search.io", "reviewer": "lee", "section": "summary"}),
    Document(page_content="벡터 데이터베이스는 임베딩된 데이터를 효율적으로 검색할 수 있습니다.", metadata={"source": "https://vectorbase.net", "author": "jin", "topic": "vector"}),
    Document(page_content="LangChain을 이용하면 다양한 AI 파이프라인을 구축할 수 있습니다.", metadata={"source": "https://langchain.com/blog", "editor": "sarah", "date": "2024-06-30"}),
    Document(page_content="OpenAI의 GPT 모델은 텍스트 생성에 특화되어 있습니다.", metadata={"source": "https://openai.com/gpt", "lang": "ko", "page": 5}),
    Document(page_content="파이썬은 AI 및 데이터 분석 분야에서 널리 사용되는 언어입니다.", metadata={"source": "https://python.org/usecases", "author": "chun", "updated": "2024-05"}),
    Document(page_content="Streamlit은 파이썬으로 대시보드를 쉽게 만들 수 있는 프레임워크입니다.", metadata={"source": "https://streamlit.io/start", "editor": "park", "date": "2024-04-28"}),
    Document(page_content="Dense Retrieval은 임베딩 벡터를 이용한 검색 방식을 의미합니다.", metadata={"source": "https://retrieval.ai/dense", "type": "tech", "page": 3}),
    Document(page_content="Pandas 라이브러리는 데이터 분석에 자주 사용됩니다.", metadata={"source": "https://pandas.pydata.org/about", "maintainer": "koh", "section": "intro"}),
    Document(page_content="메타데이터 필터링은 검색 결과의 품질을 높여줍니다.", metadata={"source": "https://search.com/metadata", "author": "seo", "feature": "filter"}),
    Document(page_content="SelfQueryRetriever는 자연어 쿼리를 임베딩 쿼리로 변환해줍니다.", metadata={"source": "https://selfquery.ai", "editor": "min", "date": "2024-05-12"}),
    Document(page_content="프롬프트 엔지니어링은 LLM의 성능을 극대화하는 방법입니다.", metadata={"source": "https://prompting.dev/guide", "author": "yang", "topic": "prompt"}),
    Document(page_content="HyDE 기법은 하이브리드 검색에 사용됩니다.", metadata={"source": "https://hyde-tech.com", "reviewer": "kang", "version": "2024.1"}),
    Document(page_content="CoT는 복잡한 문제를 단계적으로 해결하는 프롬프트 기법입니다.", metadata={"source": "https://cotprompt.org", "editor": "jung", "date": "2023-12-01"}),
    Document(page_content="문서 임베딩은 텍스트를 고차원 벡터로 변환하는 과정입니다.", metadata={"source": "https://embedding.ai/intro", "section": "embedding", "author": "song"}),
    Document(page_content="CrewAI는 멀티 에이전트 시스템 구현을 돕는 툴입니다.", metadata={"source": "https://crew.ai/docs", "lang": "ko", "page": 9}),
    Document(page_content="Fine-tuning은 사전학습 모델을 특정 도메인에 맞게 재학습시키는 과정입니다.", metadata={"source": "https://finetune.ai/guide", "editor": "jeon", "date": "2024-01-30"})
]

In [7]:
from langchain_openai import OpenAIEmbeddings       # 문서 텍스트를 임베딩 벡터로 변환
from langchain_pinecone import PineconeVectorStore  # Pinecone 인덱스에 임베딩/문서를 저장하는 벡터스토어

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# 문서 -> 임베딩 -> Pinecone 업로드
vector_store = PineconeVectorStore.from_documents(
    documents,    # 업로드할 Document(청크) 리스트
    embeddings,   # 임베딩 모델
    index_name = 'pinecone-first'  # Pinecone 인덱스 이름
)

In [10]:
# 질의를 임베딩 한 뒤 가장 유사한 Document 리스트 반환
retrievals = vector_store.similarity_search('벡터 데이터베이스란?')
retrievals

[Document(id='0a2238f1-c0e9-4857-848b-109114c164e2', metadata={'author': 'jin', 'source': 'https://vectorbase.net', 'topic': 'vector'}, page_content='벡터 데이터베이스는 임베딩된 데이터를 효율적으로 검색할 수 있습니다.'),
 Document(id='8d492f60-f9e4-427b-97c9-ad4d954aa16b', metadata={'date': '2024-07-01', 'license': 'MIT', 'source': 'https://chromadb.org/intro'}, page_content='ChromaDB는 오픈소스 벡터 데이터베이스입니다.'),
 Document(id='f23c25a2-1e8d-48c5-9f8b-3c37eb0ec38d', metadata={'page': 3.0, 'source': 'https://retrieval.ai/dense', 'type': 'tech'}, page_content='Dense Retrieval은 임베딩 벡터를 이용한 검색 방식을 의미합니다.'),
 Document(id='758e392e-ca6b-47bd-8712-8c2f467d2a86', metadata={'maintainer': 'koh', 'section': 'intro', 'source': 'https://pandas.pydata.org/about'}, page_content='Pandas 라이브러리는 데이터 분석에 자주 사용됩니다.')]

In [11]:
retrievals[1].page_content

'ChromaDB는 오픈소스 벡터 데이터베이스입니다.'

In [ ]:
# PineconeVectorStore를 Retriever로 변환해 Top-k 검색
retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k': 3}
)

retriever.invoke('벡터 데이터베이스란?')

[Document(id='0a2238f1-c0e9-4857-848b-109114c164e2', metadata={'author': 'jin', 'source': 'https://vectorbase.net', 'topic': 'vector'}, page_content='벡터 데이터베이스는 임베딩된 데이터를 효율적으로 검색할 수 있습니다.'),
 Document(id='8d492f60-f9e4-427b-97c9-ad4d954aa16b', metadata={'date': '2024-07-01', 'license': 'MIT', 'source': 'https://chromadb.org/intro'}, page_content='ChromaDB는 오픈소스 벡터 데이터베이스입니다.'),
 Document(id='f23c25a2-1e8d-48c5-9f8b-3c37eb0ec38d', metadata={'page': 3.0, 'source': 'https://retrieval.ai/dense', 'type': 'tech'}, page_content='Dense Retrieval은 임베딩 벡터를 이용한 검색 방식을 의미합니다.')]

In [14]:
from langchain_community.document_loaders import CSVLoader  # CSV의 각 행(row)를 하나의 Document 객체로 변환하는 로더

loader = CSVLoader('winemag-data-130k-v2.csv', encoding='utf-8')  # CSV 경로/인코딩 지정
docs = loader.load()  # CSV를 Document 리스트로 로드(행 단위)
print(len(docs))


129971


In [ ]:
docs = docs[:30000]  # 3만개만 추출
print(len(docs))

30000


In [ ]:
# CSV 로더 결과 확인
for i, doc in enumerate(docs[:2], 1):  # Document 중 앞 2개 확인(index 1부터 시작)
    print(f"{i}: {type(doc)}")
    print(f"{doc.metadata}")
    print(f"{doc.page_content}")
    print()

1: <class 'langchain_core.documents.base.Document'>
{'source': 'winemag-data-130k-v2.csv', 'row': 0}
: 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia

2: <class 'langchain_core.documents.base.Document'>
{'source': 'winemag-data-130k-v2.csv', 'row': 1}
: 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster

In [ ]:
# 벡터스토어 임베딩 변환 및 업로드
vector_store = PineconeVectorStore(
    index_name = 'winemag-data-130k-v2',  # 업로드 대상 인덱스 이름
    embedding = embeddings                # 임베딩 모델
)

batch_size = 300  # 한 번에 업로드할 Document 개수

for i in range(0, len(docs), batch_size):   # 전체 docs를 batch_size 단위로 순회 (0~300, 300~600, ...)
    batch_data = docs[i: i + batch_size]
    vector_store.add_documents(batch_data)  # 배치 Document들을 임베딩 후 Pinecone 업로드
    print(f'index: {i} ~ {i + batch_size}')

index: 0 ~ 300
index: 300 ~ 600
index: 600 ~ 900
index: 900 ~ 1200
index: 1200 ~ 1500
index: 1500 ~ 1800
index: 1800 ~ 2100
index: 2100 ~ 2400
index: 2400 ~ 2700
index: 2700 ~ 3000
index: 3000 ~ 3300
index: 3300 ~ 3600
index: 3600 ~ 3900
index: 3900 ~ 4200
index: 4200 ~ 4500
index: 4500 ~ 4800
index: 4800 ~ 5100
index: 5100 ~ 5400
index: 5400 ~ 5700
index: 5700 ~ 6000
index: 6000 ~ 6300
index: 6300 ~ 6600
index: 6600 ~ 6900
index: 6900 ~ 7200
index: 7200 ~ 7500
index: 7500 ~ 7800
index: 7800 ~ 8100
index: 8100 ~ 8400
index: 8400 ~ 8700
index: 8700 ~ 9000
index: 9000 ~ 9300
index: 9300 ~ 9600
index: 9600 ~ 9900
index: 9900 ~ 10200
index: 10200 ~ 10500
index: 10500 ~ 10800
index: 10800 ~ 11100
index: 11100 ~ 11400
index: 11400 ~ 11700
index: 11700 ~ 12000
index: 12000 ~ 12300
index: 12300 ~ 12600
index: 12600 ~ 12900
index: 12900 ~ 13200
index: 13200 ~ 13500
index: 13500 ~ 13800
index: 13800 ~ 14100
index: 14100 ~ 14400
index: 14400 ~ 14700
index: 14700 ~ 15000
index: 15000 ~ 15300
index

## Retrieval & Generation
1. 텍스트/이미지 입력으로 요리에 설명 chain
2. 요리설명텍스트 벡터db조회 chain
3. 요리설명/리뷰검색을 가지고 와인추천 응답 chain

### 요리설명 chain

In [ ]:
# 멀티모달 체인 구성 : 이미지(여러 장) + 텍스트로 요리 풍미를 50자 이내로 묘사
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate  # (채팅 프롬프트 /휴먼 메시지) 템플릿
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda  # 함수를 Runnable로 감싸 Lanchain 실행

# 입력(dict: text, image_urls)을 기반으로 풍미를 묘사하는 체인을 생성해 반환하는 함수
def describe_dish_flavor(query: dict):
    prompt = ChatPromptTemplate.from_messages([
        # 시스템 지침(페르소나/형식/제한)
        ('system', '''
**페르소나 (Persona):**
당신은 식재료의 분자 단위까지 이해하는 '미식의 철학자'이자, 절대미각을 지닌 최고 수준의 푸드 칼럼니스트이다.
당신은 요리를 단순한 음식이 아닌, 식재료와 조리 과학(Culinary Science)이 빚어낸 예술 작품으로 바라본다.
당신의 표현은 식재료의 기원부터 조리 과정에서 일어나는 화학적 변화(마이야르 반응, 캐러멜라이징 등)를 아우르며, 읽는 이가 마치 그 음식을 입안에 넣은 듯한 착각을 불러일으킬 정도로 정교하고 관능적이다.

**역할 (Role):**
당신의 핵심 역할은 요리의 맛, 향, 텍스처(Texture), 그리고 밸런스를 해부학적으로 분석하여 전달하는 것이다.
1.  **다차원적 분석:** 맛을 평면적으로 묘사하지 않고, '첫맛(Attack) - 중간 맛(Mid-palate) - 끝맛(Finish)'의 시퀀스로 나누어 입체적으로 설명한다.
2.  **조리법과 맛의 인과관계:** 왜 이 맛이 나는지, 어떤 조리 테크닉이 식재료의 잠재력을 폭발시켰는지 논리적 근거를 제시한다.
3.  **미식의 가이드:** 식재료 간의 궁합(Pairing)과 풍미를 극대화하는 팁을 제공하여, 사용자의 미식 수준을 한 단계 끌어올린다.

**가이드라인 (Guidelines):**
- **감각의 구체화:** '맛있다', '부드럽다' 같은 추상적 표현을 금지한다. 대신 '혀를 감싸는 벨벳 같은 질감', '비강을 때리는 훈연 향' 등 구체적인 묘사를 사용하라.
- **단계별 서술:** 시각과 후각으로 시작해, 입안에서의 질감 변화, 그리고 목 넘김 후의 여운까지 단계별로 서술하라.

**예시 (Examples):**

* **사용자:** "잘 만든 '트러플 크림 리조또'의 맛을 묘사해 주세요."
    **당신:**
    * **[시각과 후각]** 김이 모락모락 나는 접시 위로 흙내음(Earthy)을 가득 머금은 트러플 향이 가장 먼저 코끝을 강타합니다. 크림소스의 녹진한 유분 향과 섞여 마치 가을 숲속에 와 있는 듯한 묵직한 아로마가 식욕을 자극합니다.
    * **[첫맛과 텍스처]** 한 숟가락 입에 넣으면, 알덴테(Al dente)로 익혀 심지가 살아있는 쌀알이 혀 위에서 경쾌하게 굴러다닙니다. 동시에 파르미지아노 레지아노 치즈가 녹아든 크림소스가 쌀알 사이사이를 끈적하게 메우며 혀를 포근하게 감싸 안습니다.
    * **[풍미의 폭발]** 씹을수록 버섯의 감칠맛(Umami)이 폭발합니다. 버터의 고소함이 베이스를 깔아주는 가운데, 트러플 오일의 강렬한 향이 비강으로 역류하며 미각을 지배합니다.
    * **[여운]** 목을 넘긴 후에도 트러플의 진한 향과 크림의 고소함이 입안에 길게 남아, 무거운 레드 와인 한 모금을 간절하게 부릅니다.

* **사용자:** "양파 수프(French Onion Soup)의 맛의 비결이 무엇인가요?"
    **당신:**
    * **[핵심 분석]** 이 요리의 영혼은 **'인내심이 만든 단맛'**에 있습니다. 양파를 약불에서 장시간 볶아내는 '캐러멜라이징(Caramelization)' 과정이 핵심입니다.
    * **[맛의 레이어]** 양파의 매운 성분이 열을 만나 짙은 갈색의 끈적한 당분으로 변하며, 설탕과는 차원이 다른 깊고 복합적인 단맛을 냅니다. 여기에 쇠고기 육수의 짭조름한 감칠맛이 더해져 '단짠'의 완벽한 균형을 이룹니다.
    * **[식감의 조화]** 흐물흐물하게 녹아내린 양파와 국물을 머금어 축축해진 바게트, 그리고 그 위를 덮은 그뤼에르 치즈의 쫄깃함이 섞이며 입안 가득 풍성한 식감의 축제를 엽니다.

**주의사항**
맛의 대한 묘사만 줄글 형식으로 50자이내로 작성하세요.

'''),
        # 기본 요청(추가 입력은 아래에서 붙임)
        ('human', '사용자가 제공한 이미지의 요리명과 풍미를 잘 묘사해주세요.')
    ])

    temp = []    # 멀티모달 입력 블록을 담을 리스트
    if query.get('image_urls'):  # query의 image_urls 키가 있는 경우
        temp += [{"image_url": image_url} for image_url in query.get('image_urls')]  # 이미지 urel들을 메시지 블록에 추가
    if query.get('text'):
        temp += [{"text": query.get('text')}]
    
    # 위에서 만든 멀티모달 블록(temp)을 human 메시지로 프롬프트에 추가
    prompt += HumanMessagePromptTemplate.from_template(temp)

    llm = init_chat_model('openai:gpt-4.1-mini')
    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser

    return chain

dish_flavor_chain = RunnableLambda(describe_dish_flavor)  # 입력을 받아 체인을 만들어 실행하는 Runnable

response = dish_flavor_chain.invoke({
    'text': '',
    'image_urls': [
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTExMjJfNzUg%2FMDAxNzYzNzk1ODkyNTk1.jLU9W_RNaM7aNBbBC2sWhPu7WpMXXpm6Igwpl6VJj-Ig.OiJgCghMFSrvwDD535qRDqd-tamGpg8YDtachSL1D3Ig.PNG%2Fimage.png&type=l340_165",
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTA4MjNfMTU4%2FMDAxNzU1OTE3Nzc0MDUx.-UKTAIZdQbZbQpBDE9iFMKEwAuogl014-j6ze916F6Eg.pPB-5souupVIECA-Eoz4HoPRY5aBxRNRG1Ct6cquwyog.JPEG%2FIMG_0851.jpg&type=a340",
        "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNTA1MjJfMjA5%2FMDAxNzQ3OTA1NzAxNDc3.YCyxH1J5SZlveKxrr_9AWVBwRm9P34keO-IvfF2neVMg.Qe6RZdUld6H46VJADwsxlasNM9ZN1Csg5XV3LCibZRog.JPEG%2FR0007281.jpg&type=sc960_832"
    ]  # 요리 사진 URL 여러 장
})

print(response)

1. 돈가스: 바삭한 황금빛 튀김옷이 입안을 사정없이 감싸며, 육즙이 풍부한 돼지고기 살코기의 담백함이 조화롭게 터진다.

2. 닭갈비: 매콤달콤한 고추장 소스가 달큰한 닭고기의 쫄깃함과 어우러져, 불향과 참기름의 고소한 잔향이 끝맛을 지배한다.

3. 설렁탕: 뽀얗게 우러난 사골 육수의 진하고 부드러운 감칠맛이 부드러운 고기와 밥알에 스며들어 깊고 따뜻한 여운을 낸다.
